In [17]:
import sys
import os

# verify sys.path
print("sys.path:")
print("\n".join(sys.path))

# verify pythonpath
print("PYTHONPATH:")
print(os.environ.get("PYTHONPATH", ""))

sys.path:
/hdd/yihan/projects/Turbo-Rank/turbo_rank
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/hdd/yihan/projects/Turbo-Rank/.venv/lib/python3.10/site-packages
/hdd/yihan/projects/Turbo-Rank
PYTHONPATH:



In [1]:
import mlflow 
import sys 

mlflow.set_tracking_uri("http://localhost:5010")

sys.path.insert(0, "/hdd/yihan/projects/Turbo-Rank/turbo_rank")

In [ ]:
from mlflow.artifacts import download_artifacts

# URI of your registered model
model_uri = "models:/NRMS/1"
# Local directory to receive the files
dst_path  = "./NRMS_v1"

local_dir = download_artifacts(
    artifact_uri=model_uri,
    dst_path=dst_path
)
print(f"✅ Registered model artifacts downloaded to: {local_dir}")

✅ Registered model artifacts downloaded to: /hdd/yihan/projects/Turbo-Rank/notebooks/NRMS_v1/


## torch.compile

In [1]:
import mlflow
import sys
import torch

mlflow.set_tracking_uri("http://localhost:5010")

# set sys.path
sys.path.insert(0, "/hdd/yihan/projects/Turbo-Rank/turbo_rank")

model_uri = "models:/NRMS/3"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# 1. Load
model = mlflow.pytorch.load_model(model_uri, map_location=device) 
print(f"✅ Model loaded from {model_uri} to {device}")

# 2. (Optional) compile model from top level 
if torch.cuda.is_available() and not isinstance(model, torch._dynamo.OptimizedModule):
    model = torch.compile(model, mode="max-autotune", fullgraph=True)
    print("✅ Model compiled")

model.eval()


✅ Model loaded from models:/NRMS/3 to cuda:0
✅ Model compiled


OptimizedModule(
  (_orig_mod): NRMSModel(
    (news_encoder): NewsEncoder(
      (embedding): Embedding(54021, 256, padding_idx=0)
      (self_attn): _ScaledDotSelfAttention(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
      )
      (additive): Linear(in_features=256, out_features=1, bias=False)
    )
    (user_encoder): UserEncoder(
      (self_attn): _ScaledDotSelfAttention(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
      )
      (additive): Linear(in_features=256, out_features=1, bias=False)
    )
  )
)

### Sanity-check inference

In [ ]:
import torch

# -- configure dummy inputs --
B, H, L = 4, 5, 30                          # batch, history length, token length
vocab_size = 54021                         # must match your NewsEncoder
device     = "cuda:0" if torch.cuda.is_available() else "cpu"

cand = torch.randint(1, vocab_size, (B, L), device=device)
hist = torch.randint(1, vocab_size, (B, H, L), device=device)

# -- inference --
model.to(device)
with torch.inference_mode(), torch.amp.autocast(device_type='cuda'):
    scores = model(cand, hist)

print("→ scores.shape:", scores.shape)     # should be (B,)
print("→ sample scores:", scores[:B])

/hdd/yihan/projects/Turbo-Rank/.venv/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
E0503 13:37:28.035000 655424 torch/_inductor/select_algorithm.py:2100] [0/0] Runtime error during autotuning: 
E0503 13:37:28.035000 655424 torch/_inductor/select_algorithm.py:2100] [0/0] No valid triton configs. OutOfResources: out of resource: shared memory, Required: 131072, Hardware limit: 101376. Reducing block sizes or `num_stages` may help.. 
E0503 13:37:28.035000 655424 torch/_inductor/select_algorithm.py:2100] [0/0] Ignoring this choice.
E0503 13:37:28.126000 655424 torch/_inductor/select_algorithm.py:2100] [0/0] Runtime error during autotuning: 
E0503 13:37:28.126000 655424 torch/_inductor/select_algorithm.py:2100] [0/0] No valid triton configs. OutOfResource

→ scores.shape: torch.Size([4])
→ sample scores: tensor([-0.9996, -0.9948, -0.9995, -0.9984], device='cuda:0')


### Measure end-to-end latency

In [5]:
import time

with torch.inference_mode(), torch.amp.autocast(device_type='cuda'):
    # warm-up
    for _ in range(10):
        _ = model(cand, hist)

    # timed pass
    t0 = time.perf_counter()
    _  = model(cand, hist)
    torch.cuda.synchronize()          # barrier for fair timing
    t1 = time.perf_counter()

print(f"One forward pass: {(t1-t0)*1000:.2f} ms")

RuntimeError: These live storage data ptrs are in the cudagraph pool but not accounted for as an output of cudagraph trees: 

Data Pointer: 139803991480320, history: 

Data Pointer: 139803991485440, history: 


## ONNX and ONNX Runtime

In [10]:
import mlflow
import sys
import torch

mlflow.set_tracking_uri("http://localhost:5010")

# set sys.path
sys.path.insert(0, "/hdd/yihan/projects/Turbo-Rank/turbo_rank")

model_uri = "models:/NRMS/3"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# 1. Load
model = mlflow.pytorch.load_model(model_uri, map_location=device) 
print(f"✅ Model loaded from {model_uri} to {device}")

model.eval()



✅ Model loaded from models:/NRMS/3 to cuda:0


NRMSModel(
  (news_encoder): NewsEncoder(
    (embedding): Embedding(54021, 256, padding_idx=0)
    (self_attn): _ScaledDotSelfAttention(
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
    )
    (additive): Linear(in_features=256, out_features=1, bias=False)
  )
  (user_encoder): UserEncoder(
    (self_attn): _ScaledDotSelfAttention(
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
    )
    (additive): Linear(in_features=256, out_features=1, bias=False)
  )
)

In [14]:
import torch
import numpy as np
import onnxruntime as ort

# ── 1. Define & export your model (as you already do) ──────────────
B, H, L      = 4, 5, 30
vocab_size   = 54021
device       = "cuda:0" if torch.cuda.is_available() else "cpu"
cand = torch.randint(1, vocab_size, (B, L), device=device)
hist = torch.randint(1, vocab_size, (B, H, L), device=device)

torch.onnx.export(
    model,
    (cand, hist),
    "../models/onnx/nrms.onnx",
    input_names=["candidate_tokens", "history_tokens"],
    output_names=["score"],
    opset_version=18,
    dynamic_shapes={
        "candidate_tokens": {0: "batch", 1: "seq_len"},            # (B, L)
        "history_tokens": {0: "batch", 1: "hist_len", 2: "seq_len"}
    },
    dynamo=True,
)

# ── 2. Load the ONNX model ─────────────────────────────────────────
sess = ort.InferenceSession("nrms.onnx")
inp_meta = {i.name: i.shape for i in sess.get_inputs()}
print("ONNX expects symbolic dims:", inp_meta)
# e.g. {'cand': ['batch', 'token_len'], 'hist': ['batch', 'hist_len', 'token_len']}

# ── 3. Inference example A: B=2, H=4, L=50 ────────────────────────
B1, H1, L1 = 2, 4, 50
cand1 = np.random.randint(1, vocab_size, (B1, L1), dtype=np.int64)
hist1 = np.random.randint(1, vocab_size, (B1, H1, L1), dtype=np.int64)
out1 = sess.run(["score"], {"candidate_tokens": cand1, "history_tokens": hist1})[0]
print("Out1 shape:", out1.shape)  # → (2,)
print("Out1:", out1)            

# ── 4. Inference example B: B=1, H=10, L=30 ───────────────────────
B2, H2, L2 = 1, 10, 30
cand2 = np.random.randint(1, vocab_size, (B2, L2), dtype=np.int64)
hist2 = np.random.randint(1, vocab_size, (B2, H2, L2), dtype=np.int64)
out2 = sess.run(None, {"candidate_tokens": cand2, "history_tokens": hist2})[0]
print("Out2 shape:", out2.shape)  # → (1,)
print("Out2:", out2)           

W0505 16:26:53.032000 1057438 torch/onnx/_internal/exporter/_registration.py:103] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `NRMSModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NRMSModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


/hdd/yihan/projects/Turbo-Rank/.venv/lib/python3.10/site-packages/torch/onnx/_internal/exporter/_dynamic_shapes.py:265: UserWarning: # The axis name: batch will not be used, since it shares the same shape constraints with another axis: batch.
  warnings.warn(


Applied 59 of general pattern rewrite rules.
ONNX expects symbolic dims: {'candidate_tokens': ['batch', 'seq_len'], 'history_tokens': ['batch', 'hist_len', 'seq_len']}
Out1 shape: (2,)
Out1: [-0.998402  -0.9990822]
Out2 shape: (1,)
Out2: [-0.9974028]


## ONNX Quantization

In [16]:
from onnxruntime.quantization import quantize_dynamic, QuantType

quantize_dynamic(
    "../models/onnx/nrms.onnx", 
    "../models/onnx/quant/nrms.quant.onnx",
    weight_type=QuantType.QInt8
)